# Bedrock AgentCore Gateway를 사용하여 OpenAPI를 MCP Tool로 변환하기

## 개요
고객은 JSON 또는 YAML 형식의 OpenAPI 스펙을 가져와서 Bedrock AgentCore Gateway를 사용하여 MCP Tool로 변환할 수 있습니다. 이 튜토리얼에서는 API 키를 사용하여 NASA의 Open API를 호출하는 화성 날씨 Agent를 구축하는 방법을 시연합니다.

Gateway 워크플로우는 Agent를 외부 Tool에 연결하기 위해 다음 단계를 포함합니다:
* **Gateway용 Tool 생성** - REST API를 위한 OpenAPI 스펙과 같은 스키마를 사용하여 Tool을 정의합니다. OpenAPI 스펙은 Gateway 생성을 위해 Amazon Bedrock AgentCore에 의해 파싱됩니다.
* **Gateway 엔드포인트 생성** - 인바운드 인증을 통해 MCP 진입점 역할을 할 Gateway를 생성합니다.
* **Gateway에 타겟 추가** - Gateway가 특정 Tool로 요청을 라우팅하는 방법을 정의하는 OpenAPI 타겟을 구성합니다. OpenAPI 파일의 모든 API는 MCP 호환 Tool이 되며, Gateway 엔드포인트 URL을 통해 사용할 수 있게 됩니다. 각 OpenAPI Gateway 타겟에 대한 아웃바운드 인증을 구성합니다.
* **Agent 코드 업데이트** - 통합된 MCP 인터페이스를 통해 구성된 모든 Tool에 액세스하기 위해 Agent를 Gateway 엔드포인트에 연결합니다.

![How does it work](images/openapi-gateway-apikey.png)

### 튜토리얼 세부 정보


| 정보                  | 세부사항                                                   |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형          | 인터랙티브                                                  |
| AgentCore 구성요소    | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target 유형   | OpenAPI                                                   |
| Agent                | 화성 날씨 Agent                                             |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | API Key                                                   |
| LLM Model            | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| 튜토리얼 구성요소       | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 범용                                                       |
| 예제 복잡도           | 쉬움                                                       |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 AmazonCore Gateway 타겟을 생성합니다.

### 튜토리얼 아키텍처
이 튜토리얼에서는 OpenAPI yaml/json 파일에 정의된 작업을 MCP Tool로 변환하고 Bedrock AgentCore Gateway에 호스팅합니다.
시연 목적으로, 화성의 날씨와 관련된 쿼리에 답변하는 화성 날씨 Agent를 구축합니다. Agent는 NASA의 Open API를 사용합니다. 솔루션은 Amazon Bedrock Model을 사용하는 Strands Agent를 사용합니다.
예제에서는 화성 날씨를 위한 getInsightWeather Tool을 가진 매우 간단한 Agent를 사용합니다.

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook (Python 커널)
* uv
* AWS 자격 증명
* Amazon Cognito

In [ ]:
# 의존성 패키지 강제 재설치 및 업그레이드
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
# AWS_REGION 환경변수가 없으면 기본값으로 us-east-1 사용
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# Get the directory of the current script
# Jupyter 환경에서는 __file__이 정의되지 않으므로 fallback 처리
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
# 상위 디렉토리로 이동하여 utils 모듈 경로 설정
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
# Python 모듈 검색 경로에 추가 (최우선 순위)
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils

# Gateway가 사용할 IAM 역할 생성
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway 인바운드 인증을 위한 Amazon Cognito Pool 생성

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
# OAuth 2.0 scope 정의 (읽기/쓰기 권한)
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
# 토큰 요청 시 사용할 scope 문자열 (공백으로 구분)
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
# User Pool이 이미 존재하면 재사용, 없으면 생성
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

# Resource Server 생성 또는 조회
utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

# Machine-to-Machine (M2M) 클라이언트 생성 또는 조회
client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
# OIDC discovery endpoint (JWT 검증에 필요한 메타데이터 제공)
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Gateway 생성

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
# Custom JWT authorizer 설정 (Cognito 기반 인증)
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
# MCP 프로토콜을 사용하는 Gateway 생성
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with OpenAPI target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
# Gateway Target 생성 시 필요한 ID와 URL 저장
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Bedrock AgentCore Gateway를 사용하여 NASA Open API를 MCP Tool로 변환하기

NASA의 Open API에서 화성 날씨 데이터를 가져오는 화성 날씨 Agent를 만들 것입니다. [여기](https://api.nasa.gov/)에서 Nasa Insight API에 등록해야 합니다. 무료입니다! 등록하면 이메일로 API Key를 받게 됩니다. OpenAPI 타겟 생성을 위한 자격 증명 Provider를 구성할 때 API Key를 사용하세요.

In [ ]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

# NASA API 호출을 위한 API Key credential provider 생성
response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey",
    apiKey="", # Get an API key by signing up at api.nasa.gov. Takes 2-min to get an API key in your email.
)

pprint(response)
# Egress 인증에 사용할 credential provider ARN 저장
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

# OpenAPI 타겟 생성

#### S3에 NASA Open API json 파일 업로드

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
# Your s3 bucket to upload the OpenAPI json file.
# 계정 ID와 리전을 포함한 고유한 버킷 이름 생성
bucket_name = f'agentcore-gateway-{account_id}-{region}'
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# Upload the file using put_object and read response
try:
    # us-east-1은 LocationConstraint 없이 버킷 생성
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        # 다른 리전은 LocationConstraint 필수
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    # OpenAPI 스펙 파일을 S3에 업로드
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # Construct the ARN of the uploaded object with account ID and region
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### 아웃바운드 인증 구성 및 Gateway 타겟 생성

In [ ]:
# S3 Uri for OpenAPI spec file
# MCP Tool로 변환할 OpenAPI 스펙 파일의 S3 위치 지정
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# API Key credentials provider configuration
# 외부 API 호출 시 사용할 API Key 인증 설정
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # Replace this with the name of the api key name expected by the respective API provider. For passing token in the header, use "Authorization"
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # Location of api key. Possible values are "HEADER" and "QUERY_PARAMETER".
                    #"credentialPrefix": " " # Prefix for the token. Valid values are "Basic". Applies only for tokens.
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
# Gateway Target 생성 (OpenAPI를 MCP Tool로 변환)
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Strands Agent에서 Bedrock AgentCore Gateway 호출하기

Strands Agent는 Model Context Protocol (MCP) 스펙을 구현하는 Bedrock AgentCore Gateway를 통해 AWS Tool과 원활하게 통합됩니다. 이 통합은 AI Agent와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로, Bedrock AgentCore Gateway는 기본 MCP API인 ListTools와 InvokeTools를 노출하는 프로토콜 호환 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트 또는 SDK가 안전하고 표준화된 방식으로 사용 가능한 Tool을 검색하고 상호 작용할 수 있습니다. Strands Agent가 AWS 서비스에 액세스해야 할 때, 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP Authorization 스펙)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격하게 준수하여 강력한 보안과 액세스 제어를 보장합니다. 이는 Strands Agent의 모든 Tool 호출이 인증 단계를 거치며, 강력한 기능을 활성화하면서 보안을 유지한다는 것을 의미합니다.

예를 들어, Strands Agent가 MCP Tool에 액세스해야 할 때, 먼저 ListTools를 호출하여 사용 가능한 Tool을 검색한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 스펙을 구현하는 모든 클라이언트 또는 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI Agent 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.

# 인바운드 인증을 위해 Amazon Cognito에서 액세스 토큰 요청

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
# Cognito에서 OAuth 2.0 access token 요청 (Gateway 인증용)
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Bedrock AgentCore Gateway를 사용하여 NASA Open API를 호출하여 화성 날씨 Agent에게 질문하기

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

# MCP 프로토콜을 위한 HTTP transport 생성 함수
def create_streamable_http_transport():
    # Bearer token을 헤더에 포함하여 Gateway에 연결
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

# MCP 클라이언트 초기화 (Gateway와 통신)
client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
# Bedrock LLM 모델 설정
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# MCP 클라이언트 컨텍스트 관리자 사용
with client:
    # Call the listTools 
    # Gateway에서 사용 가능한 MCP Tool 목록 조회
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    # LLM 모델과 Tool을 결합한 Agent 생성
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    #print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    agent("What is the weather in northern part of the mars")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    #Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    # MCP Tool을 명시적으로 호출 (Tool 이름은 targetname___operationId 형식)
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # You can replace this with unique identifier. 
    name=targetname+"___getInsightWeather", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    arguments={"ver": "1.0","feedtype": "json"}
    )
    #Print the MCP Tool response
    print(f"Tool Call result: {result['content'][0]['text']}")


**문제: 아래 셀을 실행할 때 다음 오류가 발생하면 pydantic과 pydantic-core 버전 간의 비호환성을 나타냅니다.**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**해결 방법은?**

호환되는 pydantic==2.7.2와 pydantic-core 2.27.2가 모두 설치되어 있는지 확인해야 합니다. 완료되면 커널을 재시작하세요.

# 정리
IAM 역할, IAM 정책, Credentials Provider, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성되며, 정리 작업의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## Gateway 삭제 (선택사항)

In [ ]:
import utils
# Gateway 리소스 삭제 (정리 작업)
utils.delete_gateway(gateway_client,gatewayID)